In [1]:
# Importando a biblioteca os
import os
import sys

In [2]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
    from google.colab import drive
    drive.mount('/content/drive')

    # Instalação de requisitos
    !apt-get update # Update apt-get repository.
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
    !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
    !tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
    !pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

    # Definindo a variável de ambiente do Java
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    # Definindo a variável de ambiente do Spark
    #os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
    os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

Mounted at /content/drive
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,920 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,357 kB]
Fetched 3,510 kB in 1s (2,713 kB/s)
R

In [3]:
# Variaveis de configuração
# Diretorio base dos dados

# Local PC
#dir_base = "data/"
# Google Drive
dir_base = "/content/drive/MyDrive/jupyter/pcd_0124_analise_de_credito/data/"

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Análise de Crédito - Buerau Balance") \
    .getOrCreate()

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# Leitura dos dados - Bureau Balance

In [5]:
data_bb = spark.read.csv(dir_base + "bureau_balance.csv", header = True)
data_bb.show()

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows



# Criação de flags para auxilio na visão temporal dos dados

In [6]:
# Verifica valor máximo da coluna e gera valores baseados no intervalo
max_value = 12 # No campo DAYS_CREDIT o valor é em dias
interval = 12 # Usar intervalos em dias
list_value = [3, 6] # Valores iniciais já adicionados na lista fora do intervalo

for i in range(0, max_value+1, interval):
    list_value.append(i)
list_value.remove(0)
column_name = 'MONTHS_BALANCE'

def case_when_flags_days(list_value, column_name):
    last = len(list_value)-1
    sql_case = ""
    list_columns_flag = []
    list_sql_case = []
    for i in list_value:
        sql_case += "CASE WHEN " + column_name + " >= -" + str(i) + " THEN 1 ELSE 0 END AS ultimos_" + str(i) + "_meses"
        list_columns_flag.append("ultimos_" + str(i) + "_meses")
        if i!=list_value[last]:
            sql_case += ", "

    list_sql_case.append(sql_case)
    list_sql_case.append(list_columns_flag)

    return list_sql_case

list_case = case_when_flags_days(list_value, column_name)
list_case_query = list_case[0]
list_flags_columns_m = list_case[1]

In [7]:
# Query SQL com visão temporal
spark_query = f"""
SELECT *, {list_case_query}
    FROM data_bb
    ORDER BY SK_ID_BUREAU
"""

data_bb.createOrReplaceTempView("data_bb")

df_tmp_01 = spark.sql(spark_query)

df_tmp_01.createOrReplaceTempView("df_tmp_01")
df_tmp_01.show()

+------------+--------------+------+---------------+---------------+----------------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|ultimos_3_meses|ultimos_6_meses|ultimos_12_meses|
+------------+--------------+------+---------------+---------------+----------------+
|     5001709|           -18|     C|              0|              0|               0|
|     5001709|           -10|     C|              0|              0|               1|
|     5001709|           -17|     C|              0|              0|               0|
|     5001709|            -5|     C|              0|              1|               1|
|     5001709|            -9|     C|              0|              0|               1|
|     5001709|           -13|     C|              0|              0|               0|
|     5001709|           -16|     C|              0|              0|               0|
|     5001709|            -1|     C|              1|              1|               1|
|     5001709|            -4|     C|              0|  

In [8]:
# Query SQL com visão temporal
spark_query = """
SELECT distinct STATUS FROM df_tmp_01
"""
data_bb.createOrReplaceTempView("data_bb")

check_STATUS= spark.sql(spark_query)

check_STATUS.createOrReplaceTempView("check_STATUS")
check_STATUS.show()

+------+
|STATUS|
+------+
|     3|
|     0|
|     5|
|     C|
|     X|
|     1|
|     4|
|     2|
+------+



In [9]:
# Gera Case When pra uma lista
# check_STATUS = Dataframe com a coluna que deseja gerar o Case When
list_value = [row[0] for row in check_STATUS.collect()]
column_name = 'STATUS'

def case_when_flags_column(list_value, column_name):
    last = len(list_value)-1
    sql_case = ""
    list_columns_flag = []
    list_sql_case = []
    for i in list_value:
        sql_case += "CASE WHEN " + column_name + "='" + str(i) + "' THEN 1 ELSE 0 END AS " + column_name + "_" + str(i)
        list_columns_flag.append(column_name + "_" + str(i))
        if i!=list_value[last]:
            sql_case += ", "

    list_sql_case.append(sql_case)
    list_sql_case.append(list_columns_flag)

    return list_sql_case

list_case = case_when_flags_column(list_value, column_name)
list_case_query = list_case[0]
list_flags_columns = list_case[1]

print(list_case_query)
print(list_flags_columns)

CASE WHEN STATUS='3' THEN 1 ELSE 0 END AS STATUS_3, CASE WHEN STATUS='0' THEN 1 ELSE 0 END AS STATUS_0, CASE WHEN STATUS='5' THEN 1 ELSE 0 END AS STATUS_5, CASE WHEN STATUS='C' THEN 1 ELSE 0 END AS STATUS_C, CASE WHEN STATUS='X' THEN 1 ELSE 0 END AS STATUS_X, CASE WHEN STATUS='1' THEN 1 ELSE 0 END AS STATUS_1, CASE WHEN STATUS='4' THEN 1 ELSE 0 END AS STATUS_4, CASE WHEN STATUS='2' THEN 1 ELSE 0 END AS STATUS_2
['STATUS_3', 'STATUS_0', 'STATUS_5', 'STATUS_C', 'STATUS_X', 'STATUS_1', 'STATUS_4', 'STATUS_2']


In [10]:
# Query SQL com visão temporal
spark_query = f"""
SELECT *, {list_case_query}
    FROM df_tmp_01
    ORDER BY SK_ID_BUREAU
"""

df_tmp_01.createOrReplaceTempView("df_tmp_01")

df_tmp_02 = spark.sql(spark_query)

df_tmp_02.createOrReplaceTempView("df_tmp_02")
df_tmp_02.show()

+------------+--------------+------+---------------+---------------+----------------+--------+--------+--------+--------+--------+--------+--------+--------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|ultimos_3_meses|ultimos_6_meses|ultimos_12_meses|STATUS_3|STATUS_0|STATUS_5|STATUS_C|STATUS_X|STATUS_1|STATUS_4|STATUS_2|
+------------+--------------+------+---------------+---------------+----------------+--------+--------+--------+--------+--------+--------+--------+--------+
|     5001709|             0|     C|              1|              1|               1|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|           -20|     C|              0|              0|               0|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|            -1|     C|              1|              1|               1|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|            -2|     C|              1|

# Sumarizar na visão cliente (Automatizada)

In [11]:
print(list_flags_columns)
print(list_flags_columns_m)

['STATUS_3', 'STATUS_0', 'STATUS_5', 'STATUS_C', 'STATUS_X', 'STATUS_1', 'STATUS_4', 'STATUS_2']
['ultimos_3_meses', 'ultimos_6_meses', 'ultimos_12_meses']


In [12]:
# Importar bibliotecas
from pyspark.sql.functions import col, round, sum, avg, max, min, when, count

# Definir as colunas para uso de agregação
columns_agg_total = list_flags_columns

column_flags = list_flags_columns_m

expressions_agg = []

for flag in column_flags:
  for column in columns_agg_total:
    expressions_agg.append(round(count(when(col(flag) == 1, col(column))), 2).alias(f"QTD_{column.upper()}_{flag.upper()}"))

expressions_agg = tuple(expressions_agg)

# Aplicar as expressões de agregação
df_tmp_03 = df_tmp_02.groupBy("SK_ID_BUREAU").agg(*expressions_agg).orderBy("SK_ID_BUREAU")

# Mostrar o DataFrame resultante
df_tmp_03.show()

+------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+
|SK_ID_BUREAU|QTD_STATUS_3_ULTIMOS_3_MESES|QTD_STATUS_0_ULTIMOS_3_MESES|QTD_STATUS_5_ULTIMOS_3_MESES|QTD_STATUS_C_ULTIMOS_3_MESES|QTD_STATUS_X_ULTIMOS_3_MESES|QTD_STATUS_1_ULTIMOS_3_MESES|QTD_STATUS_4_ULTIMOS_3_MESES|QTD_STATUS_2_ULTIMOS_3_MESES|QTD_STATUS_3_ULTIMOS_6_MESES|QTD_ST

# Salvar a tabela sumarizada

In [13]:
df_tmp_03 = df_tmp_03.repartition(1)
df_tmp_03.write.mode("overwrite").csv(dir_base + "bureau_balance_agg.csv",header=True)